<a href="https://colab.research.google.com/github/NickEg72/Rep-Nick/blob/main/Baseline_NLP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline-решение

По мотивам ноутбука https://www.kaggle.com/code/hardtype/parsing-news-from-rbc-lenta-ru

## 1. Парсим новости с сайта Lenta.ru

In [89]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

In [90]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
# from IPython import display

In [91]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])

        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])

        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 37,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("lenta_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [92]:
# Задаем тут параметры
query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "0" # topic = тематика новости
dateFrom = '2023-01-01'
dateTo = "2023-12-30"

param_dict = {'query'     : query,
              'from'      : str(offset),
              'size'      : str(size),
              'dateFrom'  : dateFrom,
              'dateTo'    : dateTo,
              'sort'      : sort,
              'title_only': title_only,
              'type'      : material,
              'bloc'      : bloc,
              'domain'    : domain}

print("param_dict:", param_dict)

param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2023-01-01', 'dateTo': '2023-12-30', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '0', 'domain': '1'}


In [107]:
# Тоже будем собирать итеративно, правда можно ставить time_step побольше, т.к.
# больше лимит на запрос статей. И Работает быстрее :)

parser = lentaRu_parser()

tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 90,
                         save_every = 1000,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Parsing articles from 2023-01-01 to 2023-04-01


<ipython-input-91-174bc838a5ce>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 2023-04-02 to 2023-07-01


<ipython-input-91-174bc838a5ce>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 2023-07-02 to 2023-09-30


<ipython-input-91-174bc838a5ce>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 2023-10-01 to 2023-12-30


<ipython-input-91-174bc838a5ce>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Finish
4000


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1363803,https://lenta.ru/news/2023/01/01/exponenta/,Ким Чен Ын пообещал нарастить производство яде...,1672531825,1672531825,1,1,0,0,2,[1],https://icdn.lenta.ru/images/2023/01/01/03/202...,1672531825,Фото: ЦТАК / Reuters Марина Совина Лидер КНДР ...,Ким Чен Ын пообещал нарастить производство яде...,Фото: ЦТАК / Reuters Марина Совина Лидер ... я...
1,1363805,https://lenta.ru/news/2023/01/01/yaroslavl/,В российском городе пропал ребенок,1672532081,1672532081,1,1,0,0,1,[4],https://icdn.lenta.ru/images/2023/01/01/03/202...,1672532081,Фото: Кирилл Шипицин / РИА Новости Марина Сови...,В российском городе пропал ребенок,Фото: Кирилл Шипицин / РИА Новости ... мальчик...
2,1363807,https://lenta.ru/news/2023/01/01/alco/,Россиянам рассказали о влиянии алкоголя на сон,1672533004,1672533004,1,1,0,0,1,[2],https://icdn.lenta.ru/images/2023/01/01/03/202...,1672533004,Фото: Pixabay Марина Совина Терапевт Ирина Анд...,Россиянам рассказали о влиянии алкоголя на сон,Фото: Pixabay Марина Совина Терапевт ... употр...
3,1363808,https://lenta.ru/news/2023/01/01/ded_moroz/,Подсчитана пенсия Деда Мороза,1672533251,1672533252,1,1,0,1,1,[2],https://icdn.lenta.ru/images/2023/01/01/03/202...,1672533251,Фото: Илья Наймушин/ РИА Новости Марина Совина...,Подсчитана пенсия Деда Мороза,Фото: Илья Наймушин/ РИА Новости Марина ... Ги...
4,1363806,https://lenta.ru/news/2023/01/01/anomalia_/,Климатолог предупредил о возможных погодных ан...,1672533531,1672533531,1,1,0,0,12,[281],https://icdn.lenta.ru/images/2023/01/01/03/202...,1672533531,Фото: Komsomolskaya Pravda / Global Look Press...,Климатолог предупредил о возможных погодных ан...,Фото: Komsomolskaya Pravda / Global Look ... К...


In [108]:
tbl.to_csv("Lenta_sample.csv", index=False)

In [49]:
#tbl = pd.read_csv("Lenta_sample.csv")

In [109]:
tbl.shape

(4000, 16)

In [110]:
tbl['bloc'].value_counts(normalize=True)

1     0.21100
2     0.19650
3     0.11200
8     0.07825
4     0.07700
37    0.05875
5     0.03925
12    0.03425
7     0.03125
47    0.02975
87    0.02750
48    0.02750
6     0.02725
9     0.02300
0     0.01425
86    0.01025
49    0.00200
11    0.00025
Name: bloc, dtype: float64

Найдем соответствие между кодом блока, его названием и кодом в соревновании:

* 1 - Россия - 0
* 37 - Силовые структуры - 2
* 3 - Бывший СССР - 3
* 4 - Экономика - 1
* 5 - Наука и техника - 8
* 8 - Спорт - 4
* 48 - Туризм - 7
* 87 - Здоровье - 5

In [111]:
tbl[tbl.bloc == 3].iloc[0]

docid                                                    1363858
url                       https://lenta.ru/news/2023/01/01/kule/
title          На Украине заявили о поражении объекта инфраст...
modified                                              1672557480
lastmodtime                                           1672558031
type                                                           1
domain                                                         1
status                                                         0
part                                                           0
bloc                                                           3
tags                                                        [55]
image_url      https://icdn.lenta.ru/images/2023/01/01/10/202...
pubdate                                               1672557480
text           Фото: Gleb Garanich / Reuters Варвара Кошечкин...
rightcol       На Украине заявили о поражении объекта инфраст...
snippet        Фото: Gleb

In [112]:
tbl = tbl[tbl.bloc.isin([1, 37, 3, 4, 5, 8, 48, 87])]

TagsMap = {1 : 0, 3 : 3, 4 : 1, 5 : 8, 8 : 4, 37 : 2, 48 : 7, 87 : 5}

tbl['topic'] = tbl['bloc'].map(TagsMap)

<ipython-input-112-5f5bf2ec18e9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl['topic'] = tbl['bloc'].map(TagsMap)


In [113]:
tbl.shape

(2525, 17)

In [114]:
tbl['topic'].value_counts(normalize=True) # можно сверить с распределением меток классов в соревновании

0    0.334257
3    0.177426
4    0.123960
1    0.121980
2    0.093069
8    0.062178
5    0.043564
7    0.043564
Name: topic, dtype: float64

## 2. Машинное обучение

Загружаем данные и обучаем модель на разбиении трейн-тест

In [115]:
tbl_new = tbl[~tbl.text.isna()]

print(len(tbl), len(tbl_new))

2525 2448


In [116]:
X = tbl_new[['text']]
y = tbl_new['topic']

X.shape

(2448, 1)

In [ ]:
# использовать "вероятностные модели"

# class_weight = "balanced"

# в обучающих данных взять поровну новостей каждого класса

In [117]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [118]:
X_train.shape, X_test.shape

((1836, 1), (612, 1))

In [119]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vec = CountVectorizer() # подбор гиперпараметров очень помогает
vec.fit(X_train['text'])

bow = vec.transform(X_train['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(X_test['text'])

print(bow.shape)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))

(1836, 47374)
              precision    recall  f1-score   support

           0       0.68      0.95      0.79       182
           1       0.96      0.76      0.84        90
           2       0.91      0.85      0.88        59
           3       0.86      0.82      0.84       113
           4       1.00      0.94      0.97        64
           5       0.89      0.68      0.77        25
           7       1.00      0.49      0.66        39
           8       0.92      0.55      0.69        40

    accuracy                           0.82       612
   macro avg       0.90      0.75      0.80       612
weighted avg       0.85      0.82      0.82       612



Загружаем тестовые данные, обучаем итоговую модель и делаем прогноз.

In [121]:
Test = pd.read_csv("test_news.csv")
Test

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...
...,...
26270,Фото: РИА Новости Алевтина Запольская Главное ...
26271,Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...
26272,Фото: Олег Харсеев / Коммерсантъ Александр Кур...
26273,Владимир Зеленский Фото: Yves Herman / Reuters...


In [122]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vec = CountVectorizer()
vec.fit(X['text'])

bow = vec.transform(X['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(Test['content'])

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y)
pred = clf.predict(bow_test)

In [123]:
pred[:10], len(pred)

(array([2, 1, 4, 0, 4, 3, 2, 3, 0, 3]), 26275)

Сохраняем прогноз в файл.

In [124]:
subm = pd.read_csv("base_submission_news.csv")
subm.head()

,topic,index
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


from matplotlib import pyplot as plt
_df_0['level_0'].plot(kind='hist', bins=20, title='level_0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='level_0', y='index', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['level_0']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'level_0'}, axis=1)
              .sort_values('level_0', ascending=True))
  xs = counted['level_0']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('level_0', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('level_0')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['topic']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'topic'}, axis=1)
              .sort_values('topic', ascending=True))
  xs = counted['topic']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('topic', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('topic')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_5.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_6['level_0'].plot(kind='line', figsize=(8, 4), title='level_0')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_7['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

In [125]:
subm['topic'] = pred

subm.to_csv("bow_logreg_lenta.csv", index=False)

In [126]:
subm['topic'].value_counts(normalize=True)

0    0.520076
1    0.133815
3    0.098230
4    0.086432
2    0.071627
8    0.043273
7    0.030447
5    0.016099
Name: topic, dtype: float64